In [1]:
import numpy as np
import pandas as pd

In [2]:
df1_signal = pd.read_csv('../csv/2017/forBDTtraining/ttHToNonbb_M125_powheg/ttHToNonbb_M125_powheg_forBDTtraining_central_1.csv')
df1_signal_target = [1]*df1_signal.shape[0]
#df1_signal['mbb_loose']
df1_TT = pd.read_csv('../csv/2017/forBDTtraining/TTTo2L2Nu/TTTo2L2Nu_forBDTtraining_central_1.csv')
df1_TT_target = [0]*df1_TT.shape[0]
#df1_TT
tmp0 = [df1_signal,df1_TT]
df1 = pd.concat(tmp0)
target = df1_signal_target+df1_TT_target
df1['target'] = target

In [3]:
#train_test split
df1_train = df1.sample(frac=0.5,random_state=123)
df1_test = df1.drop(df1_train.index)

In [4]:
df1_train_target = df1_train['target'].copy()
df1_train = df1_train.drop('target',axis=1)
df1_test_target = df1_test['target'].copy()
df1_test = df1_test.drop('target',axis=1)

In [7]:
print df1_train_target.shape, df1_train.shape, df1_test.shape, df1_test_target.shape

(5957, 2) (5957, 124) (4282, 124) (4282, 2)


In [6]:
#importing KERAS
import keras
from keras.layers import Embedding, Input, Flatten, Dense, Activation
from keras.models import Model, Sequential
from keras.optimizers import SGD
from keras.utils import to_categorical


df1_train_target = keras.utils.to_categorical(df1_train_target, num_classes=2)
df1_test_target = keras.utils.to_categorical(df1_test_target, num_classes=2)

Using TensorFlow backend.


In [8]:
#defining keras model
model = Sequential()
model.add(Dense(100,activation='sigmoid',input_dim=124))
model.add(Dense(100,activation='sigmoid'))
model.add(Dense(2,activation='tanh'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               12500     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 22,802
Trainable params: 22,802
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='mean_squared_error',optimizer=SGD(),metrics=['binary_accuracy', ])

In [11]:
hist = model.fit(df1_train,df1_train_target,
                 batch_size=1000,
                 epochs=100,
                 validation_data=(df1_test,df1_test_target),
                 verbose = 1
                 )

Train on 5957 samples, validate on 4282 samples
Epoch 1/100
5957/5957 [==============================] - 0s 82us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 2/100
5957/5957 [==============================] - 0s 6us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 3/100
5957/5957 [==============================] - 0s 6us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 4/100
5957/5957 [==============================] - 0s 6us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 5/100
5957/5957 [==============================] - 0s 6us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 6/100
5957/5957 [==============================] - 0s 6us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accura

5957/5957 [==============================] - 0s 5us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 52/100
5957/5957 [==============================] - 0s 5us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 53/100
5957/5957 [==============================] - 0s 5us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 54/100
5957/5957 [==============================] - 0s 5us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 55/100
5957/5957 [==============================] - 0s 5us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 56/100
5957/5957 [==============================] - 0s 5us/step - loss: nan - binary_accuracy: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00
Epoch 57/100
5957/5957 [=================

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
